# PyCity Schools Analysis

* Your analysis here
---

In [58]:
# Dependencies and Setup
from pathlib import Path
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

In [59]:
# Calculate the Totals (Schools and Students)
school_count = school_data["school_name"].count()
student_count = student_data['student_name'].count()

# Calculate the Total Budget
total_budget = school_data['budget'].sum()

In [60]:
# Calculate the Average Scores
average_maths_score = student_data['maths_score'].mean()
average_reading_score = student_data['reading_score'].mean()

In [61]:
# Calculate the Percentage Pass Rates
passing_maths_count = school_data_complete[(school_data_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) *100

# passing_maths_reading_count = counting for students that have passed both math AND reading
passing_both_count = school_data_complete[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_both_percentage = passing_both_count / float(student_count) *100

In [62]:
# Convert to DataFrame
area_summary = pd.DataFrame({"Total Schools": [school_count],
                             "Total Students": [student_count],
                             "Total Budget": [total_budget],
                             "Average Maths Score": [average_maths_score],
                            "Average Reading Score": [average_reading_score],
                             "% Passing Math": [passing_maths_percentage],
                             "% Passing Reading": [passing_reading_percentage],
                             "% Overall Passing": [passing_both_percentage]
                             })

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

In [63]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])[["type"]]

# Calculate the total student count per school from school_data
per_school_counts = school_data.set_index(["school_name"])[["size"]]

# # Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data.set_index(["school_name"])[["budget"]]

school_types_alph = school_types.sort_index()
per_school_counts_alph = per_school_counts.sort_index()
per_school_budget_alph = per_school_budget.sort_index()

per_school_counts_cap = school_data.set_index(["school_name"])["size"]
per_school_budget_cap = school_data.set_index(["school_name"])["budget"]

per_school_capita = per_school_budget_cap / per_school_counts_cap
per_school_capita_alph = per_school_capita.sort_index()
school_cap_df = pd.DataFrame(per_school_capita_alph, columns=["Per Student Budget"])
# # Calculate the average test scores per school from school_data_complete
per_school_maths = school_data_complete.groupby('school_name')[['maths_score']].mean()
per_school_reading = school_data_complete.groupby('school_name')[['reading_score']].mean()

In [64]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.
school_passing_maths = school_data_complete[school_data_complete['maths_score'] >= 50].groupby(['school_name'])['maths_score'].count()
school_passing_maths_df = pd.DataFrame(school_passing_maths)

school_passing_reading =  school_data_complete[school_data_complete['reading_score'] >= 50].groupby(['school_name'])['reading_score'].count()
school_passing_reading_df = pd.DataFrame(school_passing_reading)

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.
passing_maths_and_reading = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)].groupby(['school_name'])['maths_score'].count()
passing_maths_and_reading_df = pd.DataFrame(passing_maths_and_reading)

In [66]:
#  Calculate the Percentage Pass Rates
per_school_passing_maths = (school_passing_maths / per_school_counts_cap *100)
per_school_passing_maths_df = pd.DataFrame(per_school_passing_maths, columns=["% Passing Maths"])

per_school_passing_reading = (school_passing_reading / per_school_counts_cap *100)
per_school_passing_reading_df = pd.DataFrame(per_school_passing_reading, columns=["% Passing Reading"])

overall_passing_rate = (passing_maths_and_reading / per_school_counts_cap *100)
overall_passing_rate_df = pd.DataFrame(overall_passing_rate, columns=["% Overall Passing"])

In [67]:
# Convert to DataFrame
per_school_sum = pd.merge(school_types_alph, per_school_counts_alph, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, per_school_budget_alph, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, school_cap_df, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, per_school_maths, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, per_school_reading, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, per_school_passing_maths_df, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, per_school_passing_reading_df, on= "school_name", how='inner')
per_school_sum = pd.merge(per_school_sum, overall_passing_rate_df, on= "school_name", how='inner')

per_school_summary = per_school_sum.rename(columns={"type": "School Type", "size": "Total Students",
                                                         "budget": "Total School Budget","maths_score": "Average Maths Score",
                                                         "reading_score": "Average Reading Score",                                                         
                                                         })
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


## Top Performing Schools (By % Overall Passing)

In [68]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=False)
top_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

In [69]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

In [70]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby("school_name")[["maths_score"]].sum()
year_nine_scores.columns=["Year 9 Maths"]

year_ten_scores = year_ten.groupby("school_name")[["maths_score"]].sum()
year_ten_scores.columns=["Year 10 Maths"]

year_eleven_scores = year_eleven.groupby("school_name")[["maths_score"]].sum()
year_eleven_scores.columns=["Year 11 Maths"]

year_twelve_scores = year_twelve.groupby("school_name")[["maths_score"]].sum()
year_twelve_scores.columns=["Year 12 Maths"]

# Combine series into single DataFrame
maths_scores_by_year = pd.merge(year_nine_scores, year_ten_scores, on= "school_name", how="inner")
maths_scores_by_year = pd.merge(maths_scores_by_year, year_eleven_scores, on= "school_name", how="inner")
maths_scores_by_year = pd.merge(maths_scores_by_year, year_twelve_scores, on= "school_name", how="inner")
# Minor data wrangling
maths_scores_by_year.index.name = None

# Display the DataFrame
maths_scores_by_year

,Year 9 Maths,Year 10 Maths,Year 11 Maths,Year 12 Maths
Bailey High School,105696,89081,90541,74710
Cabrera High School,38186,33756,34226,26971
Figueroa High School,58617,52137,48787,43051
Ford High School,57495,49126,45635,36985
Griffin High School,29771,28864,25881,20869
Hernandez High School,94787,84500,75240,64708
Holden High School,8959,8562,7379,6093
Huang High School,58305,52565,50060,40154
Johnson High School,97257,83424,82228,64853
Pena High School,19799,18099,18566,12885


## Reading Score by Year

In [71]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby("school_name")[["reading_score"]].sum()
year_nine_scores.columns=["Year 9 Reading"]

year_ten_scores = year_ten.groupby("school_name")[["reading_score"]].sum()
year_ten_scores.columns=["Year 10 Reading"]

year_eleven_scores = year_eleven.groupby("school_name")[["reading_score"]].sum()
year_eleven_scores.columns=["Year 11 Reading"]

year_twelve_scores = year_twelve.groupby("school_name")[["reading_score"]].sum()
year_twelve_scores.columns=["Year 12 Reading"]

# Combine series into single DataFrame
reading_scores_by_year = pd.merge(year_nine_scores, year_ten_scores, on= "school_name", how="inner")
reading_scores_by_year = pd.merge(reading_scores_by_year, year_eleven_scores, on= "school_name", how="inner")
reading_scores_by_year = pd.merge(reading_scores_by_year, year_twelve_scores, on= "school_name", how="inner")

# Minor data wrangling
reading_scores_by_year.index.name = None

# Display the DataFrame
reading_scores_by_year

,Year 9 Reading,Year 10 Reading,Year 11 Reading,Year 12 Reading
Bailey High School,103375,87781,87967,74217
Cabrera High School,37579,33239,34319,27449
Figueroa High School,60144,51638,49029,42900
Ford High School,57990,48844,46615,37110
Griffin High School,29459,28723,26131,20275
Hernandez High School,94636,86653,74439,64951
Holden High School,9093,8105,7551,5850
Huang High School,57958,53319,49562,40173
Johnson High School,96207,85025,83823,63641
Pena High School,19511,18081,18356,12944


## Scores by School Spending

In [72]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

In [83]:
# Create a copy of the school summary since it has the "Per Student Budget"
#  This step can be skipped but its best to make a copy.
school_spending = per_school_summary.copy()
school_spending['Per Student Budget'] = school_spending['Per Student Budget'].str.replace('[^\d]', '', regex=True).astype(float)/100


In [74]:
# Categorise spending based on the bins.
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(school_spending["Per Student Budget"], bins=spending_bins, labels=group_names, include_lowest=True)
school_spending_df = per_school_summary.copy()

In [75]:
#  Calculate averages for the desired columns.
spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Reading Score"].mean()
spending_passing_maths = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Overall Passing"].mean()

In [76]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Maths Score": spending_maths_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Maths": spending_passing_maths,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})
# Display results
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


## Scores by School Size

In [77]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [78]:
# Categorize the spending based on the bins
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=group_names, include_lowest=True)
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
school_name,,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630,Large (2000-5000)
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,<$585,Medium (1000-2000)
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645,Large (2000-5000)
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645,Large (2000-5000)
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630,Medium (1000-2000)
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,$645-680,Large (2000-5000)
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,<$585,Small (<1000)
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,$645-680,Large (2000-5000)
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766,$645-680,Large (2000-5000)


In [79]:
# Calculate averages for the desired columns.
size_maths_scores = per_school_summary.groupby(["School Size"], observed=True)["Average Maths Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"], observed=True)["Average Reading Score"].mean()
size_passing_maths = per_school_summary.groupby(["School Size"], observed=True)["% Passing Maths"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"], observed=True)["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"], observed=True)["% Overall Passing"].mean()

In [80]:
# Assemble into DataFrame
size_summary = pd.DataFrame({
    "Average Maths Score": size_maths_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Maths": size_passing_maths,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})
# Display results
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

In [81]:
# Create new series using groupby for:
# Type | Average Maths Score | Average Reading Score | % Passing Maths | % Passing Reading | % Overall Passing
type_maths_scores = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_maths = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
type_overall_passing = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [82]:
# Assemble into DataFrame
type_summary = type_summary_df = pd.DataFrame({
    "Average Maths Score": type_maths_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Maths": type_passing_maths,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": type_overall_passing
})
# Display results
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
